In [0]:
import msal
import json
import logging
import requests
import os
import base64
from datetime import datetime, timedelta
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText
from email.mime.application import MIMEApplication
from pyspark.sql import SparkSession
import pandas as pd
from pyspark.sql import functions as F
from pyspark.dbutils import DBUtils  # For secret management in Databricks

# Initialize Databricks Utilities
dbutils = DBUtils(spark)

# Microsoft 365 Application credentials fetched from Databricks secrets
CLIENT_ID = dbutils.secrets.get(scope="project-hercules", key="CLIENT_ID")
CLIENT_SECRET = dbutils.secrets.get(scope="project-hercules", key="CLIENT_SECRET")
TENANT_ID = dbutils.secrets.get(scope="project-hercules", key="TENANT_ID")
MAIL_USERNAME = 'hercules@kubrickgroup.com'

AUTHORITY = f"https://login.microsoftonline.com/{TENANT_ID}"
SCOPE = ["https://graph.microsoft.com/.default"]

# Configure root logger
logger = logging.getLogger("databricks_logger")
logger.setLevel(logging.DEBUG)

# Add a console handler to see logs in the notebook
if not logger.handlers:
    handler = logging.StreamHandler()
    handler.setLevel(logging.DEBUG)
    formatter = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s')
    handler.setFormatter(formatter)
    logger.addHandler(handler)

logger.propagate = False  # Prevent duplicate log messages

# Add handler
if not logger.handlers:
    logger.addHandler(ch)

# Initialize Spark session
spark = SparkSession.builder.appName("ProjectHercules").enableHiveSupport().getOrCreate()


In [0]:
# Read the lists of companies and excluded people from text files
def read_list_from_file(file_path):
    with open(file_path, 'r') as file:
        return [line.strip() for line in file.readlines() if line.strip()]

# Load company and recipient lists
allowed_companies = read_list_from_file('list_trading_companies.txt')
# insurance_companies = read_list_from_file('list_insurance_companies.txt')
target_companies = read_list_from_file('list_target_companies.txt')

# print(target_companies)

In [0]:
# Define the managers and their employees in a dictionary
managers_employees = {
    'jemmasmith@kubrickgroup.com': ['harrywates@kubrickgroup.com'],
    
    'benhayes@kubrickgroup.com': ['keishachristiemorgan@kubrickgroup.com','julialaming@kubrickgroup.com','riyapatel@kubrickgroup.com', 'heatherdonaldson@kubrickgroup.com', 'tobyenstone@kubrickgroup.com','mariahutcheson@kubrickgroup.com'],
    
    'nickallen@kubrickgroup.com': [ 'benhunt@kubrickgroup.com', 'joannabottinojones@kubrickgroup.com', 'matttaylor@kubrickgroup.com', 'jacksanders@kubrickgroup.com','simonhecker@kubrickgroup.com','lewisbarnes@kubrickgroup.com', 'roxannegreene@kubricsweakgroup.com'],
}

for keys,vals in managers_employees.items():
    print(keys,vals)

jemmasmith@kubrickgroup.com ['harrywates@kubrickgroup.com']
benhayes@kubrickgroup.com ['keishachristiemorgan@kubrickgroup.com', 'julialaming@kubrickgroup.com', 'riyapatel@kubrickgroup.com', 'heatherdonaldson@kubrickgroup.com', 'tobyenstone@kubrickgroup.com', 'mariahutcheson@kubrickgroup.com']
nickallen@kubrickgroup.com ['benhunt@kubrickgroup.com', 'joannabottinojones@kubrickgroup.com', 'matttaylor@kubrickgroup.com', 'jacksanders@kubrickgroup.com', 'simonhecker@kubrickgroup.com', 'lewisbarnes@kubrickgroup.com', 'roxannegreene@kubricsweakgroup.com']


In [0]:
def create_company_postings(email, start_date, end_date, company_type='target'):
    """
    Creates job postings DataFrame for either target companies or active trading companies
    Parameters:
        email (str): The email address of the user.
        start_date (str): The start date for the job postings.
        end_date (str): The end date for the job postings.
        company_type (str): The type of companies to filter ('target' or 'active_trading').

    Returns:
        pd.DataFrame: A DataFrame containing job postings.
    """
     # Determine the companies based on company_type
    if company_type == 'target':
        companies_str = "', '".join([company.replace("'", "''") for company in target_companies])
    elif company_type == 'active_trading':
        companies_str = "', '".join([company.replace("'", "''") for company in allowed_companies])
    else:
        raise ValueError("Invalid company_type. Must be 'target' or 'active_trading'.")

    # Query to fetch job postings
    query = f"""
        SELECT DISTINCT
            j.companyName AS Company,
            j.jobTitle AS Title,
            j.jobFunctions AS JobFunction,
            j.country AS Country,
            j.city AS City,
            j.seniority AS Seniority,
            j.linkedinJobPostUrl AS URL,
            j.skill AS Skill
        FROM hive_metastore.goldlayer_coresignal.recent_jobs_with_industries AS j
        JOIN hive_metastore.goldlayer_coresignal.salesforce_rivery_gold AS c
            ON LOWER(j.companyName) = LOWER(c.company_name)
        WHERE c.Email = '{email}'
          AND j.jobPostedDate BETWEEN '{start_date}' AND '{end_date}'
          AND j.country = 'United Kingdom'
          AND j.companyName IN ('{companies_str}')
    """
    
    result = spark.sql(query)  # Replace this with your actual Spark SQL execution
    df = result.toPandas()

    if df.empty:
        logger.info(f"No {company_type} job postings found for email: {email}")
        return pd.DataFrame()  # Return an empty DataFrame if no postings found
    else:
        logger.info(f"{company_type} job postings found for email: {email}")

    # Making URLs clickable
    df['URL'] = df['URL'].apply(lambda url: f'<a href="{url}">{url}</a>')
    df['Location'] = df['City'] + ', ' + df['Country']

    # Fill missing columns
    required_columns = ['Company', 'Title', 'JobFunction', 'Location', 'Seniority', 'URL', 'Skill', 'Country']
    for column in required_columns:
        if column not in df.columns:
            df[column] = ''
        df[column] = df[column].fillna('').astype(str)

    # ** Grouping by unique job postings and aggregating skills **
    df = df.groupby(['Company', 'Title', 'JobFunction', 'Location', 'Seniority', 'URL', 'Country'], as_index=False).agg({
        'Skill': lambda x: ', '.join(sorted(set(x)))  # Aggregate and join skills
    })

    # Sort by 'JobPostingsCount' in descending order and then by 'Company'
    df = df.sort_values(by=['Company'], ascending=[True]).reset_index(drop=True)

    return df

# create_company_postings('harrywates@kubrickgroup.com', '2024-11-10', '2024-11-24', 'active_trading')

In [0]:
def collate_job_postings(start_date, end_date):
    """
    Collate job postings for all managers and their employees into separate DataFrames.
    Parameters:
        start_date (str): The start date for the job postings.
        end_date (str): The end date for the job postings.
    Returns:
        dict: A dictionary with managers' emails as keys and DataFrames as values.
    """
    managers_postings = {}

    for manager_email, employees in managers_employees.items():
        all_postings = []


        # Collate target company postings
        for employee_email in employees:
            target_postings = create_company_postings(employee_email, start_date, end_date, company_type='target')
            active_trading_postings = create_company_postings(employee_email, start_date, end_date, company_type='active_trading')

            if not target_postings.empty:
                all_postings.append((employee_email, 'target', target_postings))
            if not active_trading_postings.empty:
                all_postings.append((employee_email, 'active_trading', active_trading_postings))

       # Create a summary DataFrame for the manager
        manager_summary = {
            'active_trading': pd.concat([postings for _, type_, postings in all_postings if type_ == 'active_trading'], ignore_index=True) if any(type_ == 'active_trading' for _, type_, _ in all_postings) else pd.DataFrame(),
            'target': pd.concat([postings for _, type_, postings in all_postings if type_ == 'target'], ignore_index=True) if any(type_ == 'target' for _, type_, _ in all_postings) else pd.DataFrame(),
        }

       # Sort and group the postings by company name
        for key in manager_summary.keys():
            if not manager_summary[key].empty:
                # Group by company and count postings
                company_counts = (
                    manager_summary[key]
                    .groupby('Company', as_index=False)
                    .agg(PostingsCount=('Title', 'count'))  # Count the number of postings
                )
                
                # Merge counts back to the original postings
                manager_summary[key] = (
                    manager_summary[key]
                    .merge(company_counts, on='Company', how='left')
                    .sort_values(by=['PostingsCount','Company'], ascending=[False,True])  # Sort by company name
                    .reset_index(drop=True)  # Reset index
                )


       # Ensure both target and active_trading keys exist, even if empty
        if 'target' not in manager_summary:
            manager_summary['target'] = pd.DataFrame()
        if 'active_trading' not in manager_summary:
            manager_summary['active_trading'] = pd.DataFrame()

        if all_postings:
            logger.info(f"Total postings collected for {manager_email}: {len(all_postings)}")

         # Logging the manager's summary shape
        logger.info(f"Manager {manager_email} summary - Target: {manager_summary['target'].shape[0]} rows, Active Trading: {manager_summary['active_trading'].shape[0]} rows")

        managers_postings[manager_email] = manager_summary

    return managers_postings

# Example usage
# managers_postings = collate_job_postings('2024-11-10', '2024-11-24')

# Output the results for each manager
# for manager_email, postings in managers_postings.items():
#     print(f"\nManager: {manager_email}")
#     print("Target Companies DataFrame:")
#     display(postings["target"])
#     print("\nActive Trading Companies DataFrame:")
#     display(postings["active_trading"])

In [0]:
from datetime import datetime, timedelta
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText
from email.mime.application import MIMEApplication
import smtplib
import pandas as pd

# Calculate the date range for the previous week
today = datetime.today()
last_monday = today - timedelta(days=today.weekday() + 7)
last_sunday = last_monday + timedelta(days=6)
start_date = last_monday.strftime('%Y-%m-%d')
end_date = last_sunday.strftime('%Y-%m-%d')

# Format the dates for the email
formatted_start_date = datetime.strptime(start_date, '%Y-%m-%d').strftime('%d %B %Y')
formatted_end_date = datetime.strptime(end_date, '%Y-%m-%d').strftime('%d %B %Y')

logger.info(f"Querying job postings between {start_date} and {end_date}")


# -------------------------------------------#
def df_to_html(df, title):
    """Convert a DataFrame to an HTML table grouped by Company and Country, with spacing between job postings."""
    if df.empty:
        return f"<h3>{title}</h3><p>No postings found.</p>"

    # Sort the DataFrame by Company name
    df = df.sort_values(by=['Company'], ascending=True)
    
    data = f"<h3>{title}</h3>"
    current_company = None
    current_country = None

    for _, row in df.iterrows():
        # Group by Company and Country
        if current_company != row['Company']:
            if current_company is not None:
                data += '<br><br>'  # Add space between different companies
            data += f"<b>{row['Company']} ({row['Country']})</b><br>"  # Bold the company and add country in parentheses
            current_company = row['Company']
            current_country = row['Country']  # Initialize current_country for the new company

        # Add Job Details (Title, JobFunction, Location, Seniority, URL, Skills)
        data += (f"{row['Title']} - {row['JobFunction']}; {row['Location']}, {row['Country']}; "
                 f"{row['Seniority']}; {row['URL']}; skills - {row['Skill']}<br><br>")  # Add space between jobs
    
    return data

2024-11-29 16:19:22,566 - databricks_logger - INFO - Querying job postings between 2024-11-18 and 2024-11-24


In [0]:
def create_email_body(manager_email, postings):
    """Create HTML email bodies for the manager's job postings."""

     # Query for team member's first name based on their email
    name_query = f"""
        SELECT DISTINCT First_Name
        FROM hive_metastore.goldlayer_coresignal.salesforce_rivery_gold
        WHERE Email = '{manager_email}'
    """
    result = spark.sql(name_query).collect()

    # Safely extract the first name
    if result:
        first_name = result[0].First_Name
    else:
        first_name = 'there'

    email_body = f"<p>Hi {first_name} ({manager_email}),</p>"
    email_body += f"<p>Please find below the job postings for your team from {formatted_start_date} to {formatted_end_date}:</p>"

    # Target Companies
    try:
        target_html = df_to_html(postings['target'], "Target Company Job Postings")
        email_body += target_html
    except:
        None

    # Active & Trading Companies
    try:
        active_trading_html = df_to_html(postings['active_trading'], "Active & Trading Company Job Postings")
        email_body += active_trading_html
    except:
        None

    email_body += "<p>Best Regards,<br>The Hercules Team </p>"
    return email_body

# output= create_email_body('nickallen@kubrickgroup.com@kubrickgroup.com', postings)
# print(output)

In [0]:
def get_access_token():
    """Get the access token using MSAL."""
    app = msal.ConfidentialClientApplication(
        CLIENT_ID,
        authority=AUTHORITY,
        client_credential=CLIENT_SECRET,
    )
    result = app.acquire_token_for_client(scopes=SCOPE)
    if "access_token" in result:
        return result['access_token']
    raise Exception("Failed to acquire token", result.get("error"), result.get("error_description"))

In [0]:
def send_email_via_graph_api(subject, recipient, body, attachment=None, attachment_name=None):
    recipient='jackseery@kubrickgroup.com'
    access_token = get_access_token()
    headers = {'Authorization': f'Bearer {access_token}', 'Content-Type': 'application/json'}

    email_data = {
        "message": {
            "subject": subject,
            "body": {"contentType": "HTML", "content": body},
            "from": {"emailAddress": {"address": MAIL_USERNAME}},
            "toRecipients": [{"emailAddress": {"address": recipient}}],
        }
    }

    if attachment:
        email_data["message"]["attachments"] = [
            {"@odata.type": "#microsoft.graph.fileAttachment", "name": attachment_name, "contentBytes": base64.b64encode(attachment).decode('utf-8')}
        ]

    response = requests.post(f'https://graph.microsoft.com/v1.0/users/{MAIL_USERNAME}/sendMail', headers=headers, data=json.dumps(email_data))
    if response.status_code != 202:
        raise Exception(f"Error sending email: {response.status_code} - {response.text}")
    logger.info(f"Email sent to {recipient}")

In [0]:
def send_manager_emails(managers_postings):
    """Send emails to each manager with their respective job postings."""
    for manager_email, postings in managers_postings.items():
        logger.info(f"Postings for {manager_email}: {postings.keys()}")  # Log available keys
        
        # Check if 'target' and 'active_trading' exist
        if 'target' in postings:
            subject_target = "Target Company Job Postings"
            body_target = create_email_body(manager_email,{
                'target': postings['target']
            })
            send_email_via_graph_api(subject_target, manager_email, body_target)
        else:
            logger.warning(f"No target postings for {manager_email}")

        if 'active_trading' in postings:
            subject_trading = "Active & Trading Company Job Postings"
            body_trading = create_email_body(manager_email, {
                'active_trading': postings['active_trading']
            })
            send_email_via_graph_api(subject_trading, manager_email, body_trading)
        else:
            logger.warning(f"No active trading postings for {manager_email}")

# Main execution
managers_postings = collate_job_postings(start_date, end_date)

# Replace with your actual email and password
send_manager_emails(managers_postings)

2024-11-29 16:19:23,505 - databricks_logger - INFO - target job postings found for email: harrywates@kubrickgroup.com
2024-11-29 16:19:24,043 - databricks_logger - INFO - active_trading job postings found for email: harrywates@kubrickgroup.com
2024-11-29 16:19:24,068 - databricks_logger - INFO - Total postings collected for jemmasmith@kubrickgroup.com: 2
2024-11-29 16:19:24,068 - databricks_logger - INFO - Manager jemmasmith@kubrickgroup.com summary - Target: 6 rows, Active Trading: 36 rows
2024-11-29 16:19:24,556 - databricks_logger - INFO - target job postings found for email: keishachristiemorgan@kubrickgroup.com
2024-11-29 16:19:25,069 - databricks_logger - INFO - active_trading job postings found for email: keishachristiemorgan@kubrickgroup.com
2024-11-29 16:19:25,532 - databricks_logger - INFO - No target job postings found for email: julialaming@kubrickgroup.com
2024-11-29 16:19:26,037 - databricks_logger - INFO - active_trading job postings found for email: julialaming@kubrickg